<a href="https://colab.research.google.com/github/Sion1225/Study-Deeplearning-NLP/blob/master/Character-Level%20Neura%20Machine%20Translation%20with%20Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset is from "http://www.manythings.org/anki"

#Pre-processing

In [1]:
"""
import os
import shutil
import zipfile
"""

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
"""
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)
"""

"\nhttp = urllib3.PoolManager()\nurl ='http://www.manythings.org/anki/fra-eng.zip'\nfilename = 'fra-eng.zip'\npath = os.getcwd()\nzipfilename = os.path.join(path, filename)\nwith http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       \n    shutil.copyfileobj(r, out_file)\n\nwith zipfile.ZipFile(zipfilename, 'r') as zip_ref:\n    zip_ref.extractall(path)\n"

In [3]:
lines = pd.read_csv("/content/drive/MyDrive/GitHub/Study-Deeplearning-NLP/DataSet/spa.txt", names=["src", "tar", "lic"], sep='\t')
del lines["lic"]
print("Count of all semples : ", len(lines))

Count of all semples :  139013


In [4]:
lines = lines[:80000]

In [5]:
print(lines[:5])

   src      tar
0  Go.      Ve.
1  Go.    Vete.
2  Go.    Vaya.
3  Go.  Váyase.
4  Hi.    Hola.


In [6]:
lines.sample(10)

,src,tar
20896,I'm very forgetful.,Soy muy despistado.
74607,The prisoners tried to escape.,Los prisioneros intentaron escaparse.
62247,Do you need a place to stay?,¿Necesitas un lugar en el que quedarte?
6357,Tom just died.,Tom acaba de morir.
33488,I ate chicken nuggets.,Comí deditos de pollo.
41251,Tom makes a difference.,Tom marca la diferencia.
49056,I thought you trusted me.,Creí que confiabas en mí.
4966,He is reading.,Está leyendo.
10069,I want that bag.,Quiero esa bolsa.
2230,I hate milk.,Odio la leche.


In [7]:
lines.tar = lines.tar.apply(lambda x : "\t" + x + "\n")
lines.sample(10)

,src,tar
61007,Tom swims better than I do.,\tTomás nada mejor que yo.\n
53285,How do I open this window?,\t¿Cómo abro esta ventana?\n
30867,They live downstairs.,\tViven en el piso de abajo.\n
261,Push it.,\tEmpújenlo.\n
40074,Please sweep the floor.,\tHagan el favor de barrer el suelo.\n
3123,Are you hurt?,\t¿Estás herida?\n
18662,We're defenseless.,\tEstamos indefensos.\n
54010,I think most of it's true.,\tCreo que la mayor parte es cierta.\n
60833,Tom is looking at me funny.,\tTom me está mirando de una manera rara.\n
48592,I found out where she is.,\tAverigüé dónde está ella.\n


In [8]:
src_vocab = set()
for line in lines.src :
    for char in line :
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [9]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print("Source sentences' char set :",src_vocab_size)
print("Target sentences' char set :",tar_vocab_size)

Source sentences' char set : 81
Target sentences' char set : 100


In [10]:
print(src_vocab)
print(tar_vocab)

{'H', '€', 'é', '6', 'q', "'", '$', '7', 'X', 'F', 'p', 'V', 't', 'N', '0', 'Q', 'm', '4', ' ', 'Z', '°', 'j', 'E', 'h', 'J', 'R', '/', '%', 'u', 'B', 'k', '’', 'O', 'C', 'v', 'I', '!', '3', 'c', 'S', '1', 'a', 'D', '.', 'o', 'e', 's', ',', '2', 'g', '9', '5', 'U', 'r', 'n', '"', 'b', '‘', 'w', 'x', 'K', 'G', 'P', 'd', 'l', 'Y', 'T', ':', 'W', '?', 'i', 'f', 'M', 'y', 'L', '8', 'z', 'A', '\xa0', '-'}
{'H', 'Á', '»', '€', 'é', '+', '6', '\t', 'q', '(', '7', 'á', '¡', "'", '$', 'X', ')', 'ñ', 'F', 'É', 'p', 'V', '¿', 't', 'N', '0', 'Q', 'º', 'm', '4', '—', ' ', 'Z', '°', 'j', '«', 'E', 'h', 'J', '\n', 'R', '/', '%', 'u', 'B', 'k', 'O', 'C', 'v', 'I', '!', '3', 'c', 'S', '1', 'a', 'D', '.', 'o', 'e', 's', ',', 'Ú', '2', 'g', '9', '5', 'U', 'r', 'n', '"', 'b', 'Ó', 'с', 'w', 'x', 'í', 'ó', 'K', 'ü', 'G', 'P', 'd', 'l', 'Y', 'T', ':', 'W', 'ú', '?', 'i', 'f', 'M', 'y', 'L', '8', 'z', 'A', '-'}


In [11]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[26:75])
print(tar_vocab[26:75])

['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0']
['?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v']


In [12]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'X': 46, 'Y': 47, 'Z': 48, 'a': 49, 'b': 50, 'c': 51, 'd': 52, 'e': 53, 'f': 54, 'g': 55, 'h': 56, 'i': 57, 'j': 58, 'k': 59, 'l': 60, 'm': 61, 'n': 62, 'o': 63, 'p': 64, 'q': 65, 'r': 66, 's': 67, 't': 68, 'u': 69, 'v': 70, 'w': 71, 'x': 72, 'y': 73, 'z': 74, '\xa0': 75, '°': 76, 'é': 77, '‘': 78, '’': 79, '€': 80}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, "'": 8, '(': 9, ')': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, '?': 27, 'A': 28, 'B': 29, 'C': 30, 'D': 31, 'E': 32, 'F'

In [13]:
encoder_input = []

for line in lines["src"] :
    encoded_line = []

    for char in line :
        encoded_line.append(src_to_index[char])
    
    encoder_input.append(encoded_line)

print("Source sentences : \n", lines["src"][:5])
print("Encoded source sentences : ", encoder_input[:5])

Source sentences : 
 0    Go.
1    Go.
2    Go.
3    Go.
4    Hi.
Name: src, dtype: object
Encoded source sentences :  [[29, 63, 9], [29, 63, 9], [29, 63, 9], [29, 63, 9], [30, 57, 9]]


In [14]:
decoder_input = []

for line in lines["tar"] :
  encoded_line = []

  for char in line :
    encoded_line.append(tar_to_index[char])

  decoder_input.append(encoded_line)

print("Target sentences : \n", lines["tar"][:5])
print("Encoded target sentences : ", decoder_input[:5])

Target sentences : 
 0        \tVe.\n
1      \tVete.\n
2      \tVaya.\n
3    \tVáyase.\n
4      \tHola.\n
Name: tar, dtype: object
Encoded target sentences :  [[1, 49, 58, 14, 2], [1, 49, 58, 73, 58, 14, 2], [1, 49, 54, 78, 54, 14, 2], [1, 49, 90, 78, 54, 72, 58, 14, 2], [1, 35, 68, 65, 54, 14, 2]]


In [15]:
decoder_target = []

for line in lines["tar"] :
    timestep = 0
    encoded_line = []

    for char in line :
        if timestep > 0 :
            encoded_line.append(tar_to_index[char])
        timestep += 1
    
    decoder_target.append(encoded_line)

print("Encoded target label sentences : ", decoder_target[:5])

Encoded target label sentences :  [[49, 58, 14, 2], [49, 58, 73, 58, 14, 2], [49, 54, 78, 54, 14, 2], [49, 90, 78, 54, 72, 58, 14, 2], [35, 68, 65, 54, 14, 2]]


In [16]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('Source sentences maximum length :',max_src_len)
print('Target sentences maximum length :',max_tar_len)

Source sentences maximum length : 31
Target sentences maximum length : 72


In [17]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding="post")
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [18]:
pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import ray
ray.init()

2023-02-25 16:44:14,287	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.3.0


In [20]:
#One-Hot encoding (Character-level)
@ray.remote
def One_Hot(encoder_input, decoder_input, decoder_target) :
    encoder_input = to_categorical(encoder_input)
    decoder_input = to_categorical(decoder_input)
    decoder_target = to_categorical(decoder_target)

    return encoder_input, decoder_input, decoder_target

encoder_input, decoder_input, decoder_target = ray.get(One_Hot.remote(encoder_input, decoder_input, decoder_target))

(pid=51525) 2023-02-25 16:44:18.090830: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
(pid=51525) 2023-02-25 16:44:18.090918: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
(pid=51525) 2023-02-25 16:44:18.090931: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [21]:
print(encoder_input[:3])
ray.shutdown()

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [22]:
print(encoder_input.shape)
print(max_src_len)
print(src_vocab_size)

(80000, 31, 81)
31
81


#Model

###Training

In [23]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [24]:
#Encoder
encoder_inputs = Input(shape=(max_src_len, src_vocab_size))

encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]

In [36]:
#Decoder
decoder_inputs = Input(shape=(max_tar_len, tar_vocab_size))

decoder_lstm = LSTM(units=256, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation="softmax")
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [26]:
#Model Combine & Compile
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="Nadam", loss="categorical_crossentropy", metrics=["accuracy"])

In [27]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=35, validation_split=0.2)

Epoch 1/35
1000/1000 [==============================] - 33s 23ms/step - loss: 0.7790 - accuracy: 0.7811 - val_loss: 0.8422 - val_accuracy: 0.7419
Epoch 2/35
1000/1000 [==============================] - 16s 16ms/step - loss: 0.5480 - accuracy: 0.8323 - val_loss: 0.7253 - val_accuracy: 0.7786
Epoch 3/35
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4799 - accuracy: 0.8541 - val_loss: 0.6564 - val_accuracy: 0.8005
Epoch 4/35
1000/1000 [==============================] - 16s 16ms/step - loss: 0.4349 - accuracy: 0.8686 - val_loss: 0.6071 - val_accuracy: 0.8169
Epoch 5/35
1000/1000 [==============================] - 16s 16ms/step - loss: 0.3999 - accuracy: 0.8796 - val_loss: 0.5695 - val_accuracy: 0.8278
Epoch 6/35
1000/1000 [==============================] - 16s 16ms/step - loss: 0.3721 - accuracy: 0.8881 - val_loss: 0.5415 - val_accuracy: 0.8366
Epoch 7/35
1000/1000 [==============================] - 16s 16ms/step - loss: 0.3494 - accuracy: 0.8949 - val_loss: 0.5180 -

###Activating

In [28]:
#define encoder
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [38]:
#define decoder
decoder_state_input_h = Input(shape=(256,)) # hidden
decoder_state_input_c = Input(shape=(256,)) # cell
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#decoder_lstm = LSTM(units=256, return_sequences=True)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c] # save hidden&cell state
#decoder_softmax_layer = Dense(tar_vocab_size, activation="softmax")
decoder_ouputs = decoder_softmax_layer(decoder_outputs)

decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

TypeError: ignored

In [32]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())